In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/ODU/Research_ODU/R001_Fall_2023_AI_Reproducibility/model_training
!ls

Mounted at /content/drive
/content/drive/MyDrive/ODU/Research_ODU/R001_Fall_2023_AI_Reproducibility/model_training
automated_labelling
dataset
labelled_citing_papers
lemos_R_001_M6_sentiment_predictor_001_12ep_bert.keras
lemos_R_001_sentiment_binary_002_15ep_bert.keras
model_0_predictions
model_0_predictions_for_all_42000
model_6_predictions_related_only
plots
R_001_2_label_binary_sentiment_classification_model_training.ipynb
R_001_3_label_multiclass_sentiment_classification_model_training.ipynb
R_001_binary_text_classification_related_not_related.ipynb
R_001_M6_3_class_sentiment_classification.ipynb
R_001_M7_1_binary_classification_related_not_related.ipynb
R_001_M7_2_binary_sentiment_classification.ipynb
R_001_multiclass_text_classification.ipynb
R_001_Sentiment_Analysis.ipynb
R_001_wahab.rdp


In [2]:
!pip install keras-core --upgrade
!pip install -q keras-nlp --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 75.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 91.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 33.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.16.1 which is incompatible

In [3]:
# -*- coding: utf-8 -*-
"""
R_001_Research_Project__
|
R_001_multiclass_text_classification.ipynb
Created on Thu Jan 18 16:30:03 2024
@author: Rochana Obadage
"""

import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import numpy as np
import pandas as pd
import tensorflow as tf
import keras_core as keras
import keras_nlp

import pytz
from datetime import datetime
import re
import argparse
import sys

Using TensorFlow backend


In [ ]:
!python3 -c "import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))"

2024-02-07 02:59:34.675425: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-07 02:59:34.675472: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-07 02:59:34.676640: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-07 02:59:35.840784: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Load the datasets
```for_training.csv```

```

They contain
    context -- text
    label_scored:{ -2, -1, 0, 0.5, 1 }
    labeln:{"O-NR", "P-NR", "Neutral", "Weak", "Strong"}
```t

In [4]:
!ls
!pwd

automated_labelling
dataset
labelled_citing_papers
lemos_R_001_M6_sentiment_predictor_001_12ep_bert.keras
lemos_R_001_sentiment_binary_002_15ep_bert.keras
model_0_predictions
model_0_predictions_for_all_42000
model_6_predictions_related_only
plots
R_001_2_label_binary_sentiment_classification_model_training.ipynb
R_001_3_label_multiclass_sentiment_classification_model_training.ipynb
R_001_binary_text_classification_related_not_related.ipynb
R_001_M6_3_class_sentiment_classification.ipynb
R_001_M7_1_binary_classification_related_not_related.ipynb
R_001_M7_2_binary_sentiment_classification.ipynb
R_001_multiclass_text_classification.ipynb
R_001_Sentiment_Analysis.ipynb
R_001_wahab.rdp
/content/drive/MyDrive/ODU/Research_ODU/R001_Fall_2023_AI_Reproducibility/model_training


In [5]:
df_dataset = pd.read_csv(r"../data/for_training.csv")
df_dataset

,context,label_score,label
0,"Recently, several losses [18], [24], [25] cons...",0.0,Neutral
1,The subsequent designs consider multiple negat...,0.0,Neutral
2,"the mainstream methods [1], [5], [6], [18], [6...",0.0,Neutral
3,"Accordingly, several works [24], [25] based on...",0.0,Neutral
4,The proposed method can be seamlessly implante...,0.5,Weak
...,...,...,...
1932,A Jain [125] suggests training a deep network ...,0.0,Neutral
1933,[39]12 2021 No name ICDAR PubTabNet ICDAR 2019...,0.0,Neutral
1934,The source codes of 6 papers were not executab...,-2.0,O-NR
1935,The success of DL has marked the revisiting of...,0.0,Neutral


In [6]:
df_dataset['binary_label'] = df_dataset['label_score'].apply(lambda x: 'not_related' if x == 0.0 else 'related')
df_dataset

,context,label_score,label,binary_label
0,"Recently, several losses [18], [24], [25] cons...",0.0,Neutral,not_related
1,The subsequent designs consider multiple negat...,0.0,Neutral,not_related
2,"the mainstream methods [1], [5], [6], [18], [6...",0.0,Neutral,not_related
3,"Accordingly, several works [24], [25] based on...",0.0,Neutral,not_related
4,The proposed method can be seamlessly implante...,0.5,Weak,related
...,...,...,...,...
1932,A Jain [125] suggests training a deep network ...,0.0,Neutral,not_related
1933,[39]12 2021 No name ICDAR PubTabNet ICDAR 2019...,0.0,Neutral,not_related
1934,The source codes of 6 papers were not executab...,-2.0,O-NR,related
1935,The success of DL has marked the revisiting of...,0.0,Neutral,not_related


In [7]:
df_ = df_dataset.groupby('binary_label').count()
df_.reset_index()

,binary_label,context,label_score,label
0,not_related,1756,1756,1756
1,related,181,181,181


In [8]:
df_ = df_dataset.groupby('label').count()
df_.reset_index()

,label,context,label_score,binary_label
0,Neutral,1756,1756,1756
1,O-NR,12,12,12
2,P-NR,11,11,11
3,Strong,19,19,19
4,Weak,139,139,139


In [9]:
df_dataset['target'] = df_dataset['binary_label'].apply(lambda x: 0 if x == "not_related" else 1)
df_dataset

,context,label_score,label,binary_label,target
0,"Recently, several losses [18], [24], [25] cons...",0.0,Neutral,not_related,0
1,The subsequent designs consider multiple negat...,0.0,Neutral,not_related,0
2,"the mainstream methods [1], [5], [6], [18], [6...",0.0,Neutral,not_related,0
3,"Accordingly, several works [24], [25] based on...",0.0,Neutral,not_related,0
4,The proposed method can be seamlessly implante...,0.5,Weak,related,1
...,...,...,...,...,...
1932,A Jain [125] suggests training a deep network ...,0.0,Neutral,not_related,0
1933,[39]12 2021 No name ICDAR PubTabNet ICDAR 2019...,0.0,Neutral,not_related,0
1934,The source codes of 6 papers were not executab...,-2.0,O-NR,related,1
1935,The success of DL has marked the revisiting of...,0.0,Neutral,not_related,0


In [10]:
# class balanced dataset
df_dataset_related = df_dataset[df_dataset['binary_label'] =='related']
df_dataset_not_related = df_dataset[df_dataset['binary_label'] =='not_related'].sample(n=181, replace=False)

df_balanced = pd.concat([df_dataset_related, df_dataset_not_related], ignore_index = True)
df_balanced = df_balanced.sample(frac = 1).reset_index(drop=True)

df_balanced.to_csv(r"../data/for_binary_training_balanced.csv", index=False)
df_balanced

,context,label_score,label,binary_label,target
0,We use TabStruct-Net [23] to recognize the phy...,0.5,Weak,related,1
1,We selected the Local and Global Pyramid Mask ...,0.5,Weak,related,1
2,Trying to teach computers to see and also“unde...,0.0,Neutral,not_related,0
3,To further validate the robustness of our appr...,0.5,Weak,related,1
4,"For example, [10, 11, 20, 21, 22, 23] propose ...",0.0,Neutral,not_related,0
...,...,...,...,...,...
357,2% among all published methods for this widely...,0.0,Neutral,not_related,0
358,"As aforementioned, triplet loss is a seminar e...",0.0,Neutral,not_related,0
359,"In experiment 2a, we replace the self-attentio...",0.5,Weak,related,1
360,"We use four existing recognizers —ocr [26], eq...",0.5,Weak,related,1


In [11]:
# df_b = df_balanced.groupby('binary_label').count()
# df_b.reset_index()

,binary_label,context,label_score,label,target
0,not_related,181,181,181,181
1,related,181,181,181,181


In [13]:
# # label names
# print(set(df_dataset['label']))
# print(set(df_dataset['binary_label']))
# print(set(df_dataset['target']))

{'Neutral', 'P-NR', 'Strong', 'Weak', 'O-NR'}
{'related', 'not_related'}
{0, 1}


In [14]:
# test_size =0.2
# df_train, df_test = train_test_split(df_balanced, test_size=0.2,stratify=df_balanced.label.values)

In [15]:
df_1 = df_train.groupby('binary_label').count()
df_1.reset_index()

,binary_label,context,label_score,label,target
0,not_related,144,144,144,144
1,related,145,145,145,145


In [16]:
# df_2 = df_test.groupby('binary_label').count()
# df_2.reset_index()

,binary_label,context,label_score,label,target
0,not_related,37,37,37,37
1,related,36,36,36,36


In [18]:
# df_train.to_csv(r"../data//binary_train.csv",index=False)
# df_test.to_csv(r"../data/binary_test.csv",index=False)

In [20]:
df_train_for_5_fold = pd.read_csv("../data/for_binary_training_balanced.csv")

print('Training Set Shape = {}'.format(df_train_for_5_fold.shape))
print('Training Set Memory Usage = {:.2f} MB'.format(df_train_for_5_fold.memory_usage().sum() / 1024**2))

Training Set Shape = (362, 5)
Training Set Memory Usage = 0.01 MB


In [25]:
df_train_for_5_fold

,context,label_score,label,binary_label,target
0,We use TabStruct-Net [23] to recognize the phy...,0.5,Weak,related,1
1,We selected the Local and Global Pyramid Mask ...,0.5,Weak,related,1
2,Trying to teach computers to see and also“unde...,0.0,Neutral,not_related,0
3,To further validate the robustness of our appr...,0.5,Weak,related,1
4,"For example, [10, 11, 20, 21, 22, 23] propose ...",0.0,Neutral,not_related,0
...,...,...,...,...,...
357,2% among all published methods for this widely...,0.0,Neutral,not_related,0
358,"As aforementioned, triplet loss is a seminar e...",0.0,Neutral,not_related,0
359,"In experiment 2a, we replace the self-attentio...",0.5,Weak,related,1
360,"We use four existing recognizers —ocr [26], eq...",0.5,Weak,related,1


In [26]:
BATCH_SIZE = 16
NUM_TRAINING_EXAMPLES = df_train_for_5_fold.shape[0]
TRAIN_SPLIT = 1
VAL_SPLIT = 0
STEPS_PER_EPOCH = int(NUM_TRAINING_EXAMPLES)*TRAIN_SPLIT // BATCH_SIZE

EPOCHS = 2 #8
AUTO = tf.data.experimental.AUTOTUNE
NUM_TRAINING_EXAMPLES

362

# Load a DistilBERT model from Keras NLP

In [28]:
# Load a DistilBERT model.
preset= "distil_bert_base_en_uncased"

# Use a shorter sequence length.
preprocessor = keras_nlp.models.DistilBertPreprocessor.from_preset(preset,
                                                                   sequence_length=512,
                                                                   )
# preprocessor_4_tweets
# Pretrained classifier.
classifier = keras_nlp.models.DistilBertClassifier.from_preset(preset,
                                                               preprocessor = preprocessor,
                                                               num_classes=2)

# vgg_model.add(tf.keras.layers.Dense(10, activation = 'softmax'))
classifier.summary()

100%|██████████| 580/580 [00:00<00:00, 1.14MB/s]
100%|██████████| 226k/226k [00:00<00:00, 368kB/s]
100%|██████████| 515/515 [00:00<00:00, 401kB/s]
100%|██████████| 253M/253M [00:13<00:00, 19.1MB/s]


Preprocessor: "distil_bert_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ distil_bert_tokenizer (DistilBertTokenizer)        │                                              30,522 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "distil_bert_classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ distil_bert_backbone          │ (None, None, 768)         │      66,362,880 │ padding_mask[0][0],        │
│ (DistilBertBackbone)          │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ get_item (GetItem)            │ (None, 768)               │               0 │ distil_bert_backbone[0][0] │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ pooled_dense (Dense)          │ (None, 768)               │         590,592 │ get_item[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ output_dropout (Dropout)      │ (None, 768)               │               0 │ pooled_dense[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ logits (Dense)                │ (None, 2)                 │           1,538 │ output_dropout[0][0]       │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 66,955,010 (255.41 MB)

 Trainable params: 66,955,010 (255.41 MB)

 Non-trainable params: 0 (0.00 B)

# Train your own model, fine-tuning BERT

In [ ]:
from sklearn.model_selection import KFold
from keras_nlp.models import DistilBertClassifier
from sklearn.metrics import classification_report
import numpy as np


fold_accuracies = []
fold_reports = []
fold_matrices = []

kfold = KFold(n_splits=5, shuffle=True)

for train_index, test_index in kfold.split(df_train_for_5_fold):

    # Get the training and test data for the current fold
    X_train, X_test = df_train_for_5_fold.iloc[train_index], df_train_for_5_fold.iloc[test_index]
    y_train, y_test = X_train['target'], X_test['target']


    classifier = keras_nlp.models.DistilBertClassifier.from_preset(preset,
                                                               preprocessor = preprocessor,
                                                               num_classes=2)

    # uncomment below if error
    # classifier.fit(x=X_train, y=y_train, batch_size=2)

    # Re-compile (e.g., with a new learning rate)
    classifier.compile(
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=tf.keras.optimizers.Adam(1e-5),
        metrics= ["accuracy"],
    )


    history = classifier.fit(x=X_train['context'],
                            y=y_train,
                            batch_size=BATCH_SIZE,
                            epochs=EPOCHS,
                            )

    # Evaluate the model on the test data
    y_pred = classifier.predict(X_test['context'])

    # Calculate the accuracy
    accuracy = np.mean(np.argmax(y_pred, axis=1) == y_test)
    print(f"Accuracy: {accuracy}")

    print('\n\n')
    label_to_score = {'related':1,"not_related":0}
    print('classifiation report : Cross validation')
    print(classification_report(y_test, np.argmax(y_pred, axis=1),target_names= label_to_score.keys()))


    # Calculate the accuracy
    accuracy = np.mean(np.argmax(y_pred, axis=1) == y_test)
    fold_accuracies.append(accuracy)

    # Generate classification report
    report = classification_report(y_test, np.argmax(y_pred, axis=1),target_names= label_to_score.keys())
    # report = classification_report(y_test, y_pred, output_dict=True)
    fold_reports.append(report)

    # # Generate confusion matrix
    matrix = confusion_matrix(y_test, np.argmax(y_pred, axis=1))
    fold_matrices.append(matrix)

# Print the average accuracy across all folds
print("Average Accuracy:", np.mean(fold_accuracies))

# Print the classification reports for each fold
for i, report in enumerate(fold_reports):
    print(f"Fold {i+1} Classification Report:")
    print(report)

# Print the confusion matrices for each fold
for i, matrix in enumerate(fold_matrices):
    print(f"Fold {i+1} Confusion Matrix:")
    print(matrix)

/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:37: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:37: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids


Epoch 1/2
19/19 [==============================] - 36s 873ms/step - loss: 0.6810 - accuracy: 0.5952
Epoch 2/2
3/3 [==============================] - 3s 450ms/step
Accuracy: 0.0



classifiation report : Training
              precision    recall  f1-score   support

     related       0.81      0.81      0.81        36
 not_related       0.81      0.81      0.81        37

    accuracy                           0.81        73
   macro avg       0.81      0.81      0.81        73
weighted avg       0.81      0.81      0.81        73



/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:37: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:37: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids


Epoch 1/2
19/19 [==============================] - 38s 879ms/step - loss: 0.6898 - accuracy: 0.5260
Epoch 2/2
3/3 [==============================] - 3s 422ms/step
Accuracy: 0.0



classifiation report : Training
              precision    recall  f1-score   support

     related       0.84      0.89      0.86        35
 not_related       0.89      0.84      0.86        38

    accuracy                           0.86        73
   macro avg       0.86      0.86      0.86        73
weighted avg       0.86      0.86      0.86        73



/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:37: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:37: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids


Epoch 1/2
19/19 [==============================] - 40s 890ms/step - loss: 0.6876 - accuracy: 0.5690
Epoch 2/2
3/3 [==============================] - 3s 424ms/step
Accuracy: 0.0



classifiation report : Training
              precision    recall  f1-score   support

     related       0.76      0.95      0.84        37
 not_related       0.92      0.69      0.79        35

    accuracy                           0.82        72
   macro avg       0.84      0.82      0.82        72
weighted avg       0.84      0.82      0.82        72



/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:37: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:37: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids


Epoch 1/2
19/19 [==============================] - 37s 885ms/step - loss: 0.6856 - accuracy: 0.5345
Epoch 2/2
3/3 [==============================] - 4s 400ms/step
Accuracy: 0.0



classifiation report : Training
              precision    recall  f1-score   support

     related       0.66      0.89      0.76        35
 not_related       0.84      0.57      0.68        37

    accuracy                           0.72        72
   macro avg       0.75      0.73      0.72        72
weighted avg       0.75      0.72      0.72        72



/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:37: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:37: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids


Epoch 1/2
19/19 [==============================] - 37s 882ms/step - loss: 0.6881 - accuracy: 0.5552
Epoch 2/2
3/3 [==============================] - 5s 410ms/step
Accuracy: 0.0



classifiation report : Training
              precision    recall  f1-score   support

     related       0.97      0.82      0.89        38
 not_related       0.82      0.97      0.89        34

    accuracy                           0.89        72
   macro avg       0.90      0.89      0.89        72
weighted avg       0.90      0.89      0.89        72

Average Accuracy: 0.0
Fold 1 Classification Report:
              precision    recall  f1-score   support

     related       0.81      0.81      0.81        36
 not_related       0.81      0.81      0.81        37

    accuracy                           0.81        73
   macro avg       0.81      0.81      0.81        73
weighted avg       0.81      0.81      0.81        73

Fold 2 Classification Report:
              precision    recall  f1-score   suppor

In [ ]:
saved_model_path = 'lemos_R_001_M71_relativity_predictor_001_2ep_bert.keras'
classifier.save(saved_model_path)

In [ ]:
model_path = 'lemos_R_001_M71_relativity_predictor_001_2ep_bert.keras'
reloaded_model_22 = tf.keras.models.load_model(model_path)

/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/task.py:47: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/task.py:47: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:37: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:37: UserWarning: 

# Predictions using the model M7.1

In [ ]:
import glob
import os


FOLDER_NAME = r'../data/related_binary_labelled/'

if not os.path.exists(FOLDER_NAME):
    os.makedirs(FOLDER_NAME)


label_file_list = [i for i in glob.glob("../data/Citing_Paper_contexts_csv/*.csv")]
count = 0

for file_ in label_file_list:
    df_ = pd.read_csv(file_)
    print(len(df_))

    predictions_df = df_.copy()

    predictions_df['target_predict'] = 0
    x_test_series = predictions_df['text']

    predictions = reloaded_model_22.predict(x_test_series)
    predictions_df["target_predict"] = np.argmax(predictions, axis=1)

    label_to_score = {'related':1,"not_related":0}
    mapping_reverse = {0: 'not_related', 1: 'related'}


    predictions_df['target_predict_label'] = predictions_df["target_predict"].map(mapping_reverse)
    full_filename = os.path.join(FOLDER_NAME, os.path.basename(file_))
    print(full_filename)
    print("\n\n\n")
    # print(predictions_df)

    predictions_df.to_csv(full_filename,index=False)
    print(full_filename + ' saved')